In [ ]:
# Importando as bibliotecas.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Introdução

Esta competição de recrutamento tem como objetivo realizar a previsão das vendas semanais de cada departamento das 45 lojas Walmart utilizando as bases de dados store, features train e test diponibilizadas pela empresa. Sendo assim, construirei a solução para o desafio levando em conta o contexo dos dados disponibilizados e comparando com meus conhecimentos em situações reais.

Ao longo da solução farei algumas simplificações para facilitar o entendimento e colocarei uma breve descrição dos ojetivos de cada código ao longo desse notebook. 

# Importação os Dados

Iniciaremos nossa análise exportando as bases de dados e verificando algumas características básicas.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Importando os dados.
sampleSubmission = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
features = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
X_train_full = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
y_test_full = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')

In [ ]:
# Verificando o formato de submissão.
sampleSubmission.head()

In [ ]:
# Verificando a base de lojas.
stores.head()

In [ ]:
# Verificando a base de variáveis.
features.head()

In [ ]:
# Verificando a base de treino.
X_train_full.head()

In [ ]:
# Verificando a base de teste.
y_test_full.head()

In [ ]:
# Verificando os tamanhos das bases.
sampleSubmission.shape, stores.shape, features.shape, X_train_full.shape, y_test_full.shape

# Tratamento e limpeza

### Realizando os merges (joins) entre as bases

Após analisar as caracterísicas de cada base de dados vamos realizar três merges (ou joins como são mais conhecidos) entre as bases:
- Merge entre features e stores, gerando a base features_stores_base;
- Merge entre X_train_full e features_stores_base, gerando a base X_train_base;
- Merge entre X_train_base e features_stores_base, gerando a base y_test_base.

In [ ]:
# Merge entre features e stores.
features_stores_base = features.merge(stores, on = ['Store'])

# Merge entre X_train_full e features_stores_base.
X_train_base = X_train_full.merge(features_stores_base, on = ['Store', 'Date', 'IsHoliday'])

# Merge entre X_train_base e features_stores_base.
y_test_base = y_test_full.merge(features_stores_base, on = ['Store', 'Date', 'IsHoliday'])

In [ ]:
# Verificando os dados da base features_stores_base.
features_stores_base.head()

In [ ]:
# Verificando os dados da base X_train_base.
X_train_base.head()

In [ ]:
# Verificando os dados da base y_test_base.
y_test_base.head()

In [ ]:
# Verificando o tamanho das bases geradas pelos marges.
features_stores_base.shape, X_train_base.shape, y_test_base.shape

### Incluindo novas colunas

Após realizar os marges vamos dividir a coluna Data da bases X_train_base e y_test_base nas colunas Day, Month, Year e Week. Veja que a coluna Data agora pode também ser retirada do nosso modelo, pois já temos suas informações divididas nas quatro novas colunas criadas.

In [ ]:
# Adicionando as colunas Year, Mouth, Day e Week à base X_train_base.
X_train_base['Year'] = pd.to_datetime(X_train_base['Date']).dt.year
X_train_base['Month'] = pd.to_datetime(X_train_base['Date']).dt.month
X_train_base['Day'] = pd.to_datetime(X_train_base['Date']).dt.day
X_train_base['Week'] = pd.to_datetime(X_train_base['Date']).dt.week

# Adicionando as colunas Year, Mouth, Day e Week à base y_test_base.
y_test_base['Year'] = pd.to_datetime(y_test_base['Date']).dt.year
y_test_base['Month'] = pd.to_datetime(y_test_base['Date']).dt.month
y_test_base['Day'] = pd.to_datetime(y_test_base['Date']).dt.day
y_test_base['Week'] = pd.to_datetime(y_test_base['Date']).dt.week

# Dropando a variável Date das duas bases.
X_train_base = X_train_base.drop(columns=['Date'])
y_test_base = y_test_base.drop(columns=['Date'])

In [ ]:
# Verificando as colunas criadas.
X_train_base.head()

In [ ]:
# Verificando as colunas criadas.
y_test_base.head()

### Tratando valores faltantes

Agora vamos analisar  os valores faltantes (NaN) de nossas bases de dados X_train_base e y_test_base.

In [ ]:
X_train_base.isnull().sum()

In [ ]:
y_test_base.isnull().sum()

Vejam que na base X_train_base apenas as colunas MarkDown possuem valores faltantes, já na base y_test_base, além das colunas MarkDown, as colunas CPI e Unemplyment também possuem valores vazios. Para os valores faltantes das colunas MarkDown podemos substituí-los por zero, visto que são relacionados a descontos promocionais que não acontecem o ano todo. Mas cabe observar aqui que não conhecemos como esses descontos são aplicados e uma analise de como isso acontece seria importante em um segundo momento. Já para as colunas CPI e Unemplyment vamos analizar o box plot a seguir para subsidiar nossa decisão.

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10, 5))
sns.boxplot(data = X_train_base[['CPI']], orient = "v");

fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10, 5))
sns.boxplot(data = X_train_base[['Unemployment']], orient = "v");

Observe que a variável CPI possui um comportamento próximo de uma normal padão, já a variável Unemplyment, embora também aparente ter um comportamento de uma normal padão, possui alguns outliers. Mas como também não conhecemos todas as características da variável Unemplyment para julgar seus outlies não trataremos deles nesse momento. Agora como ambas as variáveis aparentam ter comportamentos dentro do esperado podemos pensar então em substituir seus valores faltantes pela média.

In [ ]:
# Tratando dados vazios na base X_train_base.
X_train_base['MarkDown1'] = X_train_base['MarkDown1'].fillna(0)
X_train_base['MarkDown2'] = X_train_base['MarkDown2'].fillna(0)
X_train_base['MarkDown3'] = X_train_base['MarkDown3'].fillna(0)
X_train_base['MarkDown4'] = X_train_base['MarkDown4'].fillna(0)
X_train_base['MarkDown5'] = X_train_base['MarkDown5'].fillna(0)

# Tratando dados vazios na base y_test_base.
y_test_base['MarkDown1'] = y_test_base['MarkDown1'].fillna(0)
y_test_base['MarkDown2'] = y_test_base['MarkDown2'].fillna(0)
y_test_base['MarkDown3'] = y_test_base['MarkDown3'].fillna(0)
y_test_base['MarkDown4'] = y_test_base['MarkDown4'].fillna(0)
y_test_base['MarkDown5'] = y_test_base['MarkDown5'].fillna(0)
y_test_base['CPI'] = y_test_base['CPI'].fillna(y_test_base['CPI'].mean())
y_test_base['Unemployment'] = y_test_base['Unemployment'].fillna(y_test_base['Unemployment'].mean())

In [ ]:
X_train_base.isnull().sum()

In [ ]:
y_test_base.isnull().sum()

#### Tratando os tipos de variáveis

Finalizando um primeiro tratamento em nossa base de dados vamos transfomar todas as variáveis não numéricas em numéricas. Para isso veja que as colunas IsHoliday e Type são, respectivamente, dos tipos bool e object. Dessa maneira vamos substituir False e True da coluna IsHoliday por 0 e 1. Já na coluna Type substituiremos A, B e C por 3, 2 e 1.

In [ ]:
X_train_base.info()

In [ ]:
y_test_base.info()

In [ ]:
# Fazer o Label Encoding das variáveis IsHoliday e Type.
X_train_base['IsHoliday'] = X_train_base.IsHoliday.astype(int)
y_test_base['IsHoliday'] = y_test_base.IsHoliday.astype(int)
X_train_base['Type'] = X_train_base.Type.map({'A': 3, 'B': 2, 'C': 1})
y_test_base['Type'] = y_test_base.Type.map({'A': 3, 'B': 2, 'C': 1})

In [ ]:
# Verificando os encodings.
X_train_base.head()

In [ ]:
# Verificando os encodings.
y_test_base.head()

In [ ]:
X_train_base.info()

In [ ]:
y_test_base.info()

# Análise Exploratória dos Dados

Da tabela abaixo não percebo, inicialmente, nada muito fora do padão, embora existam valores negativos em algumas variáveis como, por exemplo, Weekly_Sales, isso pode ser apenas um prejuízo para aquele período não sendo possível agora cocluir se é realmente um valor atípico.

In [ ]:
# Algumas características gerais das variáveis.
X_train_base.describe()

Do gráfico a seguir, que mostra a média de vendas semanais ao longo das semanas para os anos de 2010, 2011 e 2012, veja que de fato existe um aumento significativo das médias de vendas, principalmente nas semanas do Thanksgiving e do Christmas. O que pode indicar que essas datas poderiam ser analisadas isoladamente.

In [ ]:
weekly_sales_2010 = X_train_base[X_train_base.Year == 2010].groupby('Week')['Weekly_Sales'].mean()
weekly_sales_2011 = X_train_base[X_train_base.Year == 2011].groupby('Week')['Weekly_Sales'].mean()
weekly_sales_2012 = X_train_base[X_train_base.Year == 2012].groupby('Week')['Weekly_Sales'].mean()

plt.figure(figsize = (25,10))
plt.plot(weekly_sales_2010.index, weekly_sales_2010.values)
plt.plot(weekly_sales_2011.index, weekly_sales_2011.values)
plt.plot(weekly_sales_2012.index, weekly_sales_2012.values)

plt.xticks(np.arange(1, 53, step = 1), fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel('Week of Year', fontsize = 20, labelpad = 20)
plt.ylabel('Sales', fontsize = 20, labelpad = 20)

plt.title("Average Weekly Sales - Per Year", fontsize = 24)
plt.legend(['2010', '2011', '2012'], fontsize = 20);

Na tabela de correlações a seguir observe que existem algumas variáveis com correlações altas como nos casos de MarkDown4 e MarkDown1, no entanto ambas tem quase a mesma correlação com a variável Weekly_Sales. O mesmo acontece entre as variáveis Year e Fuel_Price e Week e Mouth, nesse sentido ainda não é possível cocluir que alguma dessas variáveis pudesse ser descartada aqui para evitar problemas futuros de multicolinearidade.

In [ ]:
plt.figure(figsize=(20, 10))
heatmap = sns.heatmap(X_train_base.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
plt.title('Correlations', fontsize = 20);

O gráfico a seguir representa a média de vendas semanais em cada uma das lojas e perceba que, aparentemente, elas são bem distribuidas e não existem grandes diferenças entre lojas do mesmo tamanho. Nesse sentido, os dados parem estar relativamente balanceados.

In [ ]:
stores_sales = X_train_base.groupby('Store')['Weekly_Sales'].mean()
plt.figure(figsize=(20, 10))
sns.barplot(x = stores_sales.index, y = stores_sales.values)
plt.title("Average Weekly Sales - Per Store", fontsize = 20);

Para verificar se de fato os dados estão razoavelmente balanceados o gráfico à seguir mostra que existe uma proporcionalidade bem clara entre as médiad de vendas de lojas pequenas, médias e grandes.

In [ ]:
stores_sales = X_train_base.groupby('Type')['Weekly_Sales'].mean()
plt.figure(figsize=(10, 5))
sns.barplot(x = stores_sales.index, y = stores_sales.values)
plt.title("Average Weekly Sales - Per Type", fontsize = 20);

Nos Box Plots a seguir podemos ter uma noção de como estão distribuidos os dados de algumas variáveis (excluidas as variáveis de alta variabilidade) da base X_train_base. Observe que a distribuição de nenhuma das variáveis está fora do esperado ou diferente do que já foi analizado até aqui.

In [ ]:
# Plotando os box plots para algumas variáveis.
fig, ax = plt.subplots(4, 3, figsize = (20, 20))
sns.boxplot(data = X_train_base[['Store']], orient = "v", ax = ax[0, 0])
sns.boxplot(data = X_train_base[['Dept']], orient = "v", ax = ax[0, 1])
sns.boxplot(data = X_train_base[['Temperature']], orient = "v", ax = ax[0, 2])
sns.boxplot(data = X_train_base[['Fuel_Price']], orient = "v", ax = ax[1, 0])
sns.boxplot(data = X_train_base[['CPI']], orient = "v", ax = ax[1, 1])
sns.boxplot(data = X_train_base[['Unemployment']], orient = "v", ax = ax[1, 2])
sns.boxplot(data = X_train_base[['Type']], orient = "v", ax = ax[2, 0])
sns.boxplot(data = X_train_base[['Size']], orient = "v", ax = ax[2, 1])
sns.boxplot(data = X_train_base[['Year']], orient = "v", ax = ax[2, 2])
sns.boxplot(data = X_train_base[['Month']], orient = "v", ax = ax[3, 0])
sns.boxplot(data = X_train_base[['Day']], orient = "v", ax = ax[3, 1])
sns.boxplot(data = X_train_base[['Week']], orient = "v", ax = ax[3, 2]);

# Construindo os modelos

### Separando os dados de treino e validação

Neste momento vamos separar nossa base X_train_base em duas bases: umas com as variáveis previsoras e outra com nossa variável alvo.

In [ ]:
# Verificando as colunas da base X_train_base
X_train_base.columns

In [ ]:
# Selecionando as variáveis e criando nossas bases de previsores (forecasters) e alvo (target).
X_train_forecasters = X_train_base[['Store', 'Dept', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'Year', 'Month', 'Day', 'Week']]
y_train_target = X_train_base[['Weekly_Sales']]

In [ ]:
# Conferindo a base X_train_forecasters.
X_train_forecasters.head()

In [ ]:
# Conferindo a base y_train_target.
y_train_target.head()

In [ ]:
# Verificando os tamanhos das bases X_train_forecasters e y_train_target.
X_train_forecasters.shape, y_train_target.shape

Agora vamos dividir nossas bases de previsores e alvo em bases de treino e teste, considerando uma proporção de 75% para treino e 25% para teste.

In [ ]:
# Dividindo as bases X_train_forecasters e y_train_target para treinamento e validação dos modelos.
X_train, X_test, y_train, y_test = train_test_split(X_train_forecasters, y_train_target, test_size = 0.25, random_state = 0)

In [ ]:
# Verificando os tamanhos das bases divididas.
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Selecionando e treinando os modelos

Para realizar nossas provisões vamos trabalhar com três modelos classicos: Linear Regression, Decission Tree Regressor e Random Forest Regressor.

In [ ]:
# Treinando o modelo Linear Regression.
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

# Treinando o modelo Decision Tree Regressor.
decision_tree_regressor = DecisionTreeRegressor(random_state = 0)
decision_tree_regressor.fit(X_train, y_train)

# Treinando o modelo Random Forest Regressor.
random_forest_regressor = RandomForestRegressor(random_state = 0)
random_forest_regressor.fit(X_train, y_train)

### Validação dos modelos

Após o teinamento dos nossos modelos vamos validá-los utilizando nossas bases de teste e o erro absoluto médio ponderado WMAE (Weighted Mean Absolute Error), métrica recomendada pela competição.

In [ ]:
# Fazendo as previsões do modelo Linear Regression para a base X_test.
predictions_linear_regression = linear_regression.predict(X_test)

# Fazendo as previsões do modelo Decision Tree Regressor para a base X_test.
predictions_decision_tree_regressor = decision_tree_regressor.predict(X_test)

# Fazendo as previsões do modelo Random Forest Regressor para a base X_test.
predictions_random_forest_regressor = random_forest_regressor.predict(X_test)

In [ ]:
# Calculando o WMAE para o modelo Linear Regression.
weights = X_test.IsHoliday.apply(lambda x: 5 if x else 1)
wmae_linear_regression = mean_absolute_error(y_test, predictions_linear_regression, sample_weight = weights)

# Calculando o WMAE para o modelo Decision Tree Regressor.
weights = X_test.IsHoliday.apply(lambda x: 5 if x else 1)
wmae_decision_tree_regressor = mean_absolute_error(y_test, predictions_decision_tree_regressor, sample_weight = weights)

# Calculando o WMAE para o modelo Random Forest Regressor.
weights = X_test.IsHoliday.apply(lambda x: 5 if x else 1)
wmae_random_forest_regressor = mean_absolute_error(y_test, predictions_random_forest_regressor, sample_weight = weights)

# Exibindo os resultados.
print("O WMAE para o modelo Linear Regression é:", wmae_linear_regression)
print("O WMAE para o modelo Decision Tree Regression é:", wmae_decision_tree_regressor)
print("O WMAE para o modelo Random Forest Regressor é:", wmae_random_forest_regressor)

Veja que o modelo que apresentou o melhor resultado foi o Linear Regression com um WMAE = 14784.357169017618 e, portanto, nesse momento, será o nosso modelo base para as previsões.

# Realizando a previsão

Por fim, sendo o modelo Linear Regression nosso escolhido, vamos realizar a previsão e submissão da solução para esta competição.

In [ ]:
# Fazendo a previsão para a base y_test_base.
predictions = linear_regression.predict(y_test_base)

In [ ]:
# Criando a base de submissão.
sampleSubmission['Weekly_Sales'] = predictions
sampleSubmission.head()

In [ ]:
# Salvando a base de submissão.
sampleSubmission.to_csv('sampleSubmissionPredictions.csv',index = False)

# Considerações finais

Ao longo desse notebook, como o período para submissão era curto, tentei utilizar soluções objetivas para resolver os problemas. Tentei não exagerar nas transformações e trabalhar com técnicas simples.

Iniciei explorando as bases de dados para entender como elas se relacionavam e ajustei os parâmetros de algumas variáveis. Em seguida utilizei alguns gráficos e técnicas visuais para identificar um possível desbalanceanto nas bases, outliers ou multicolinearidades. Inicialmente coclui que não existiam tantos problemas entre as variáveis que fossem tão evidentes naquele momento e parti para a escolha e treinamento dos modelos.

Como o problema a ser resolvido era de previsão de vendas escolhi três algoritmos clássicos baseados em regreção. Após a validação o algoritmo escolhido para a previsão foi o Linear Regression com um WMAE = 14784.357169017618, que quando submetido ao Kaggle obteve um score privado de 19555.66288 um score publico de 19162.36667, o que colocaria o modelo entre os mais bem avaliados da competição.

Por fim, gostria de destacar que o modelo ainda está bem simples e exige bastante analise e reflexão para aprimorá-lo. 